In [1]:
from mand.core import Entity, node, getNode
from mand.core import ObjectDb, Timestamp, Context, SimplificationContext, _tr
from mand.core import ProfileMonitor, SummaryMonitor
from mand.core import find, addFootnote
from mand.core import DependencyManager, setDependencyManager
from mand.demos.trading import makeWorld, bookSomeTrades, PnLExplainReport
from mand.lib.dbsetup import setUpDb

db = ObjectDb()
setUpDb(db)

In [2]:
small = False

with db:
    pWorld = makeWorld(small=small)
    
pAll, bExt, bExt2 = pWorld.children()
p1 = pAll.children()[0]
p2 = pAll.children()[1]
p4 = pAll.children()[3]

b2 = p2.children()[0]
b4 = p4.children()[0]

makeWorld, TopOfTheHouse is: <Entity:/Global/TradingPortfolio/TopOfTheHouse>
    # books: 100
    # children: 10


In [3]:
ts0, ts1, ts2, ts3, ts4, ts5, eod, ts6 = bookSomeTrades(pWorld, small=small)

p1.setChildren(p1.children() + [b2])
ts7 = Timestamp()

# Sanity check of a computation we might want to optimize [Test]

The PnL report is a good test, but at 152K getValues, it's a bit too big to immediately use:

In [4]:
db3 = db.copy()
p = db3._get(pAll.meta.path())

r = PnLExplainReport(valuable=p, ts1=eod, ts2=ts7)

with ProfileMonitor():
    with SummaryMonitor():
        r.run()

# PnL explain for TopOfTheHouse: 6256.00

**Caveat: this report encountered problems. See footnotes at bottom.**

|Activity|PnL|
|-|-|
|Starting balance breaks|0.00
|prior day amends: MarketData|0.00
|prior day amends: Portfolio|0.00
|prior day amends: Trading|-4.00
|activity: MarketData|5240.00
|activity: Portfolio|1020.00
|activity: Trading|0.00
|Ending balance breaks|0.00

1. Inadequate cash discounting model used
1. Book appears multiple times
  * /Global/TradingBook/Eq-Inst0

# Compute activity summary (22.46 seconds of wall clock time)

|key|value|
|-|-|
|Context|11
|Db.Get|2176
|GetValue|152158
|GetValue/Calc|2891


### Profile by nodes.
* times are in microseconds
* cumT is total time spent in funtion
* calcT is time spent in function, but not in a child node

|fn|n|cumT|calcT|cumT/call|sys|
|-|-|-|-|-|-|
|PnLExplainReport:data|1|22,449,635|19|22,449,635|GetValue
|PnLExplainReport:data|1|22,449,615|686|22,449,615|GetValue/Calc
|TradingContainer:NPV|11|22,376,711|203|2,034,246|GetValue
|TradingContainer:NPV|11|22,376,508|123|2,034,228|GetValue/Calc
|Portfolio:items|121|22,171,768|1,714|183,237|GetValue
|Portfolio:items|121|22,171,656|1,110|183,236|GetValue/Calc
|Workbook:items|1,104|21,008,972|15,743|19,029|GetValue
|Workbook:items|1,100|20,993,228|12,498|19,084|GetValue/Calc
|PnLExplainReport:cutoffs|1|11,543,650|20|11,543,650|GetValue
|PnLExplainReport:cutoffs|1|11,543,629|205|11,543,629|GetValue/Calc
|Root:Clocks|548|11,543,322|2,239,610|21,064|Context
|TradeOpenEvent|1,014|4,509,989|4,509,989|4,447|Db.Get
|TradeOpenEvent:ticket|22,286|4,138,741|147,261|185|GetValue
|TradingTicket|1,014|3,991,480|3,991,480|3,936|Db.Get
|Root:End|278|1,446,378|1,349,106|5,202|Context
|Root:Activity MarketData|255|1,305,902|1,215,540|5,121|Context
|Root:Amend MarketData|255|1,285,317|1,189,135|5,040|Context
|Root:Activity Portfolio|263|1,281,527|1,162,222|4,872|Context
|Root:Amend Portfolio|255|1,261,650|1,169,385|4,947|Context
|Root:Start|270|1,168,035|1,081,174|4,326|Context
|Root:Activity Trading|263|1,114,315|1,019,430|4,236|Context
|Portfolio:children|242|1,107,820|2,303|4,577|GetValue
|Portfolio:children|121|1,105,516|1,376|9,136|GetValue/Calc
|Root:Amend Trading|255|1,024,055|936,663|4,015|Context
|Root:Start breaks|255|1,013,402|925,431|3,974|Context
|PortfolioUpdateEvent:children|121|437,659|4,138|3,617|GetValue
|TradingBook|102|395,210|395,210|3,874|Db.Get
|Equity:NPV|15|201,777|206|13,451|GetValue
|Equity:NPV|15|201,570|137|13,438|GetValue/Calc
|MarketInterface:spot|15|190,697|183|12,713|GetValue
|MarketInterface:spot|15|190,514|130|12,700|GetValue/Calc
|ExternalRefData:state|15|180,186|167|12,012|GetValue
|ExternalRefData:state|15|180,018|139|12,001|GetValue/Calc
|RefData:state|15|179,372|249|11,958|GetValue
|RefData:state|15|179,123|153|11,941|GetValue/Calc
|_WorkItemEvent:book1|22,286|136,767|136,767|6|GetValue
|_WorkItemEvent:book2|22,286|134,369|126,870|6|GetValue
|Clock:cutoffs|2,480|127,159|13,287|51|GetValue
|TradeOpenEvent:quantity|22,286|122,692|122,692|5|GetValue
|Clock:cutoffs|20|113,970|238|5,698|GetValue/Calc
|Clock:parent|20|112,609|251|5,630|GetValue
|Clock:parent|20|112,358|201|5,617|GetValue/Calc
|_WorkItemEvent:item|11,143|98,088|89,236|8|GetValue
|TradeOpenEvent:action|11,143|93,160|93,160|8|GetValue
|TradeOpenEvent:premium|11,143|86,056|82,657|7|GetValue
|Event:amends|11,340|75,490|75,490|6|GetValue
|TradeOpenEvent:unitPrice|11,143|74,453|74,453|6|GetValue
|TradingBook:clock|2,200|73,105|27,717|33|GetValue
|PortfolioUpdateEvent|12|56,728|56,728|4,727|Db.Get
|TradingPortfolio|10|45,809|45,809|4,580|Db.Get
|TradingBook:clock|1,100|45,388|15,228|41|GetValue/Calc
|RefDataUpdateEvent|9|41,375|41,375|4,597|Db.Get
|Clock|5|19,942|19,942|3,988|Db.Get
|Portfolio:clock|242|14,430|3,416|59|GetValue
|Portfolio:clock|121|11,013|1,880|91|GetValue/Calc
|Equity:refdata|15|10,059|181|670|GetValue
|Portfolio:books|231|9,945|2,232|43|GetValue
|Equity:refdata|15|9,877|144|658|GetValue/Calc
|MarketInterface:source|15|9,261|159|617|GetValue
|MarketInterface:source|15|9,101|144|606|GetValue/Calc
|MarketInterface|2|9,045|9,045|4,522|Db.Get
|Equity|2|8,852|8,852|4,426|Db.Get
|ClockEvent|2|8,495|8,495|4,247|Db.Get
|Portfolio:books|121|8,160|1,169|67|GetValue/Calc
|MarketDataSource|2|7,644|7,644|3,822|Db.Get
|ClockEvent:parent|8|7,251|100|906|GetValue
|RootClock|1|4,677|4,677|4,677|Db.Get
|MarketDataSource:clock|30|4,416|346|147|GetValue
|MarketDataSource:clock|15|4,070|200|271|GetValue/Calc
|ForwardCashflow|1|3,398|3,398|3,398|Db.Get
|Entity:clock|40|1,166|474|29|GetValue
|Entity:clock|20|691|254|34|GetValue/Calc
|MarketInterface:sourceName|15|462|182|30|GetValue
|ForwardCashflow:NPV|11|435|171|39|GetValue
|RefDataUpdateEvent:data|58|394|394|6|GetValue
|RootClock:cutoffs|53|352|275|6|GetValue
|MarketInterface:sourceName|15|279|190|18|GetValue/Calc
|ForwardCashflow:NPV|11|263|106|23|GetValue/Calc
|Equity:assetName|15|118|118|7|GetValue
|RootClock:cutoffs|1|77|40|77|GetValue/Calc
|RootClock:cosmicAll|1|21|12|21|GetValue
|CosmicAll:dbState|1|14|8|14|GetValue
|PnLExplainReport:valuable|2|11|11|5|GetValue
|PnLExplainReport:ts2|2|10|10|5|GetValue
|RootClock:cosmicAll|1|9|9|9|GetValue/Calc
|PnLExplainReport:ts1|2|9|9|4|GetValue
|CosmicAll:dbState|1|5|5|5|GetValue/Calc

# Caching/Reusing results

Until now, we have just checked to see if the current context contains a value for a node we are asking for, and if so, reuse that.

A better approach to caching bound *fn* on *object* in *context0* is:
    
1. Has fn been tweaked in context0? If so, return that
2. Is fn sufficiently trivial that we can avoid managing it as a node?
  * If trivial, just treat it as a pure python function and call it
  * Meta-data (inputs, outputs, etc) will be given to its caller and callees as appropriate
3. Ask object for context1
  * *context1* is a simplified version of *context0*
  * The default case is just to return *context1*
  * IRL, this could actually be a list of contexts or a pattern to match contexts against
4. Is fn cached in *context1*? If so, use that
5. Compute *fn* in *context1*
6. Construct *context2* from the inputs of the computed value
7. If *context2* is a subset of *context1*, cache *fn* in *context1*
8. Something odd happened
  * Footnote the problem as part of computation notes on the node's metadata
  * Maybe cache the node anyway in *context2*
    * Perhaps *object* will return *context2* as a siplification for future computations?
9. Return the value

Notes:
* Parallel compute of nodes not considered yet
* Context simplification (step 3) and input simplification (step 9) are probably intimately related
* The split between calculation and caching is nice:
  * BAs can write business logic
  * Computer scientists can add caching logic where needed
* We can use a profiler type object to gather runtime compute cost infomation
  * The resulting trace can be used as input to drive step 2 and step 3
    

# Graph Simplification

* Very simple input management simplifier: ignore inputs if they are leaves and not tweakable
* Simple context simplification: just switch to a simple context for certain known methods

In [5]:
class DM1(DependencyManager):
    
    def __init__(self):
        self.contexts = {}
        #self.simpleClockMethods = ('Workbook:items', 'RefData:state', 'Portfolio:children', 'Portfolio:items')
        super(DM1, self).__init__()
        
    def addDep(self, input, output):
        if not input.key.tweakable:
            if not input.inputs:
                return
        output.inputs.add(input)
        input.outputs.add(output)

    def calculated(self, node):
        if not node.isSimplified:
            return True
        ok = True
        for input in node.floatingTweakPoints():
            addFootnote(text='context simplification failure', info='%s in %s on %s in %s' % (str(node.key), node.ctx.name, str(input.key), input.ctx.name))
            ok = False
        return ok

    def simplify(self, key):
        if key.fullName() == 'RefData:state':
            obj = key.object()
            return (obj.clock().cutoffs,)
        if key.fullName() == 'Portfolio:items':
            obj = key.object()
            obj2 = obj.getObj(_tr.Clock, 'Trading')
            return (obj.clock().cutoffs, obj2.cutoffs)
        if key.fullName() == 'Workbook:items':
            obj = key.object()
            return (obj.clock().cutoffs,)
        
    def getNode(self, ctx, key):
        n = ctx._get(key)
        if n:
            return n
        s = self.simplify(key)
        if s: 
            nodes = [ getNode(bm) for bm in s ]
            values = [ bm() for bm in s ]
            k = [ (n.object(), n.key.fullName(), v) for n, v in zip(nodes, values) ]
            cKey = tuple(k)
            
            if cKey not in self.contexts:
                tweaks = dict( [ (bm, v) for bm, v in zip(s, values)])
                name = 'simp-%s' % len(self.contexts)
                ctx1 = SimplificationContext(tweaks, name)
                self.contexts[cKey] = ctx1
            ctx1 = self.contexts[cKey]
            ret = ctx1.get(key)
            ret.isSimplified = True
        else:
            ret = super(DM1, self).getNode(ctx, key)
            ret.isSimplified = False
        return ret
    
dm = DM1()
setDependencyManager(dm)


In [6]:
db4 = db.copy()
valuable = db4.get(pAll.meta.path())

r = PnLExplainReport(valuable=valuable, ts1=eod, ts2=ts7)
with ProfileMonitor():
    with SummaryMonitor():
        r.run()

# PnL explain for TopOfTheHouse: 6256.00

**Caveat: this report encountered problems. See footnotes at bottom.**

|Activity|PnL|
|-|-|
|Starting balance breaks|0.00
|prior day amends: MarketData|0.00
|prior day amends: Portfolio|0.00
|prior day amends: Trading|-4.00
|activity: MarketData|5240.00
|activity: Portfolio|1020.00
|activity: Trading|0.00
|Ending balance breaks|0.00

1. Inadequate cash discounting model used
1. Book appears multiple times
  * /Global/TradingBook/Eq-Inst0

# Compute activity summary (14.03 seconds of wall clock time)

|key|value|
|-|-|
|Context|22
|Db.Get|2176
|GetValue|43372
|GetValue/Calc|1531


### Profile by nodes.
* times are in microseconds
* cumT is total time spent in funtion
* calcT is time spent in function, but not in a child node

|fn|n|cumT|calcT|cumT/call|sys|
|-|-|-|-|-|-|
|PnLExplainReport:data|1|14,025,542|16|14,025,542|GetValue
|PnLExplainReport:data|1|14,025,526|1,017|14,025,526|GetValue/Calc
|TradingContainer:NPV|11|13,945,812|161|1,267,801|GetValue
|TradingContainer:NPV|11|13,945,650|150|1,267,786|GetValue/Calc
|Portfolio:items|61|13,721,186|814|224,937|GetValue
|Portfolio:items|55|13,721,093|607|249,474|GetValue/Calc
|Workbook:items|502|12,871,161|5,621|25,639|GetValue
|Workbook:items|300|12,865,540|3,014|42,885|GetValue/Calc
|PnLExplainReport:cutoffs|1|12,328,405|25|12,328,405|GetValue
|PnLExplainReport:cutoffs|1|12,328,379|293|12,328,379|GetValue/Calc
|Root:Clocks|59|12,327,836|123,723|208,946|Context
|simp-0|144|11,041,001|61,785|76,673|Context
|simp-1|200|10,466,011|1,027,464|52,330|Context
|TradeOpenEvent|1,014|5,172,436|5,172,436|5,101|Db.Get
|TradeOpenEvent:ticket|6,078|4,176,960|49,600|687|GetValue
|TradingTicket|1,014|4,127,360|4,127,360|4,070|Db.Get
|Root:Amend Trading|11|1,488,131|636|135,284|Context
|simp-8|144|1,487,237|63,536|10,328|Context
|simp-9|200|1,415,975|1,327,474|7,079|Context
|simp-3|144|1,050,038|62,288|7,291|Context
|simp-4|200|980,539|895,036|4,902|Context
|Portfolio:children|110|761,834|1,317|6,925|GetValue
|Portfolio:children|55|760,517|688|13,827|GetValue/Calc
|PortfolioUpdateEvent:children|55|443,309|2,325|8,060|GetValue
|TradingBook|102|403,454|403,454|3,955|Db.Get
|Equity:NPV|15|145,811|219|9,720|GetValue
|Equity:NPV|15|145,592|143|9,706|GetValue/Calc
|MarketInterface:spot|15|134,804|269|8,986|GetValue
|MarketInterface:spot|15|134,535|171|8,969|GetValue/Calc
|ExternalRefData:state|15|122,639|251|8,175|GetValue
|ExternalRefData:state|15|122,388|200|8,159|GetValue/Calc
|Clock:cutoffs|1,365|120,157|7,237|88|GetValue
|Clock:cutoffs|20|113,026|208|5,651|GetValue/Calc
|Clock:parent|20|111,313|298|5,565|GetValue
|Clock:parent|20|111,014|214|5,550|GetValue/Calc
|Root:Amend Portfolio|11|75,228|692|6,838|Context
|simp-7|144|74,283|68,324|515|Context
|Root:Activity Portfolio|18|70,557|1,492|3,919|Context
|simp-10|144|68,487|62,765|475|Context
|RefData:state|15|66,668|113|4,444|GetValue
|RefData:state|4|66,555|39|16,638|GetValue/Calc
|PortfolioUpdateEvent|12|63,627|63,627|5,302|Db.Get
|TradingPortfolio|10|46,946|46,946|4,694|Db.Get
|RefDataUpdateEvent|9|45,995|45,995|5,110|Db.Get
|_WorkItemEvent:book2|6,078|44,308|34,892|7|GetValue
|simp-5|4|41,433|11,610|10,358|Context
|TradingBook:clock|1,102|40,107|15,315|36|GetValue
|_WorkItemEvent:item|3,039|39,425|28,136|12|GetValue
|_WorkItemEvent:book1|6,078|37,844|37,844|6|GetValue
|TradeOpenEvent:quantity|6,078|33,390|33,390|5|GetValue
|TradeOpenEvent:premium|3,039|27,706|23,300|9|GetValue
|TradeOpenEvent:action|3,039|25,349|25,349|8|GetValue
|TradingBook:clock|800|24,792|9,421|30|GetValue/Calc
|Root:End|33|23,607|22,561|715|Context
|Clock|5|23,570|23,570|4,714|Db.Get
|Root:Start|26|21,873|21,187|841|Context
|Event:amends|3,122|20,630|20,630|6|GetValue
|simp-2|2|20,567|4,054|10,283|Context
|TradeOpenEvent:unitPrice|3,039|20,177|20,177|6|GetValue
|Equity|2|11,288|11,288|5,644|Db.Get
|ClockEvent:parent|8|11,191|143|1,398|GetValue
|MarketInterface:source|15|10,680|190|712|GetValue
|MarketInterface:source|15|10,489|163|699|GetValue/Calc
|Equity:refdata|15|9,502|262|633|GetValue
|Equity:refdata|15|9,240|204|616|GetValue/Calc
|MarketDataSource|2|8,934|8,934|4,467|Db.Get
|ClockEvent|2|8,574|8,574|4,287|Db.Get
|MarketInterface|2|8,002|8,002|4,001|Db.Get
|Portfolio:clock|171|7,298|1,718|42|GetValue
|RootClock|1|5,606|5,606|5,606|Db.Get
|Portfolio:clock|66|5,579|749|84|GetValue/Calc
|Root:Amend MarketData|11|5,421|663|492|Context
|MarketDataSource:clock|23|5,302|290|230|GetValue
|MarketDataSource:clock|19|5,011|203|263|GetValue/Calc
|Portfolio:books|105|4,945|1,151|47|GetValue
|simp-6|2|4,514|4,427|2,257|Context
|ForwardCashflow|1|4,405|4,405|4,405|Db.Get
|Portfolio:books|55|3,998|696|72|GetValue/Calc
|Root:Activity Trading|18|3,326|2,325|184|Context
|Root:Activity MarketData|11|1,586|1,151|144|Context
|Entity:clock|40|1,152|473|28|GetValue
|Root:Start breaks|11|745|530|67|Context
|Entity:clock|20|679|277|33|GetValue/Calc
|MarketInterface:sourceName|15|638|224|42|GetValue
|RootClock:cutoffs|53|447|251|8|GetValue
|ForwardCashflow:NPV|11|441|177|40|GetValue
|MarketInterface:sourceName|15|413|301|27|GetValue/Calc
|ForwardCashflow:NPV|11|263|106|23|GetValue/Calc
|RootClock:cutoffs|1|195|129|195|GetValue/Calc
|Equity:assetName|15|149|149|9|GetValue
|RefDataUpdateEvent:data|15|138|138|9|GetValue
|RootClock:cosmicAll|1|36|20|36|GetValue
|CosmicAll:dbState|1|30|19|30|GetValue
|PnLExplainReport:ts1|2|27|27|13|GetValue
|RootClock:cosmicAll|1|15|15|15|GetValue/Calc
|PnLExplainReport:valuable|2|15|15|7|GetValue
|PnLExplainReport:ts2|2|11|11|5|GetValue
|CosmicAll:dbState|1|10|10|10|GetValue/Calc

In [7]:
for k, ctx in sorted(dm.contexts.items(), key=lambda x: x[1].name):
    print
    print ctx.name
    for t in k:
        print '    ', t
    for t in ctx.tweaks:
        print '        ', t


simp-0
     (<Entity:/Global/Clock/Portfolio>, 'Clock:cutoffs', <TS:t=2017-03-29T12:32:23.437793,v=2017-03-29T12:32:23.437791>)
     (<Entity:/Global/Clock/Trading>, 'Clock:cutoffs', <TS:t=2017-03-29T12:32:23.437793,v=2017-03-29T12:32:23.437791>)
         <Node: /Global/Clock/Portfolio.(Clock:cutoffs)() in simp-0 @4538614032>
         <Node: /Global/Clock/Trading.(Clock:cutoffs)() in simp-0 @4538615312>

simp-1
     (<Entity:/Global/Clock/Trading>, 'Clock:cutoffs', <TS:t=2017-03-29T12:32:23.437793,v=2017-03-29T12:32:23.437791>)
         <Node: /Global/Clock/Trading.(Clock:cutoffs)() in simp-1 @4538617488>

simp-10
     (<Entity:/Global/Clock/Portfolio>, 'Clock:cutoffs', <TS:t=2017-03-29T12:32:23.506764,v=2017-03-29T12:32:23.506760>)
     (<Entity:/Global/Clock/Trading>, 'Clock:cutoffs', <TS:t=2017-03-29T12:32:23.506764,v=2017-03-29T12:32:23.437791>)
         <Node: /Global/Clock/Trading.(Clock:cutoffs)() in simp-10 @4825840912>
         <Node: /Global/Clock/Portfolio.(Clock:cutoffs)()